In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F

import sys

sys.path.insert(1,"/home/sbulusu/qcd_ml/neural_networks/libs/")
import custom_torch_net_class_lightning

sys.path.insert(1,"/home/sbulusu/qcd_ml/neural_networks/libs/pytorch-conv4d/")
from conv4d import *


In [111]:
i = -2
i_max = 5
np.mod(i, i_max)

3

In [2]:
a = np.arange(0,60)
a = a.reshape((5,4,3))
#np.pad(a,2,"wrap")

torch_a = torch.tensor(a)
torch_a.size()
print(torch_a)

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],

        [[12, 13, 14],
         [15, 16, 17],
         [18, 19, 20],
         [21, 22, 23]],

        [[24, 25, 26],
         [27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[36, 37, 38],
         [39, 40, 41],
         [42, 43, 44],
         [45, 46, 47]],

        [[48, 49, 50],
         [51, 52, 53],
         [54, 55, 56],
         [57, 58, 59]]])


In [273]:
stride = 2
row_list = range(0,len(torch_a), stride)

torch_a[row_list]

tensor([[[ 0,  1,  2],
         [ 3,  4,  5],
         [ 6,  7,  8],
         [ 9, 10, 11]],

        [[24, 25, 26],
         [27, 28, 29],
         [30, 31, 32],
         [33, 34, 35]],

        [[48, 49, 50],
         [51, 52, 53],
         [54, 55, 56],
         [57, 58, 59]]])

In [271]:
torch_a.narrow()

tensor([[ 0,  1,  2,  3],
        [ 8,  9, 10, 11],
        [16, 17, 18, 19]])

In [113]:
torch_b = torch_a.narrow(0,0,2)
print(torch_b)

tensor([[0, 1, 2, 3],
        [4, 5, 6, 7]])


In [114]:
torch.cat((torch_a,torch_b), 0)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19],
        [ 0,  1,  2,  3],
        [ 4,  5,  6,  7]])

In [ ]:
torch_a.repeat(3,3)

In [7]:
torch_a = torch.arange(0,20)
torch_a = torch_a.reshape(5,4)
print(torch_a)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])


In [13]:
torch_a.expand(5,6)

RuntimeError: The expanded size of the tensor (6) must match the existing size (4) at non-singleton dimension 1.  Target sizes: [5, 6].  Tensor sizes: [5, 4]

In [260]:
shape = [2,3,4]
n_max = np.prod(shape)
torch_range = torch.tensor(np.arange(n_max).reshape(shape))
print(torch_range)
print(torch_range.shape)
torch_range = torch_range.reshape(-1,*shape[2:])
print(torch_range)
print(torch_range.shape)
torch_range = torch_range.view(shape)
print(torch_range)
print(torch_range.shape)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])
torch.Size([2, 3, 4])
tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19],
        [20, 21, 22, 23]])
torch.Size([6, 4])
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]],

        [[12, 13, 14, 15],
         [16, 17, 18, 19],
         [20, 21, 22, 23]]])
torch.Size([2, 3, 4])


In [116]:


#conv4d_layer = conv4d.Conv4d()

In [117]:
"""Circular Padding"""

class CircularPadding(nn.Module):
    
    def __init__(self, padding=1):
        super(CircularPadding, self).__init__()
        
        #self.padding = padding
        #batch axis
        self.padding = np.concatenate(([0],padding))
        
        self.dims = len(self.padding)

            
    def forward(self,x):
        
        padded_x = x
        
        #for d in dims:
        for d in range(1,self.dims):
            #print(d)
            
            pad_l = self.padding[d]
            if self.padding[d] != 0:
                end_strip = padded_x.narrow(d, 0, int(pad_l)).clone()
                start_strip = padded_x.narrow(d, int(-pad_l), int(pad_l)).clone()
                
                #print(padded_x.size())
                #print(end_strip.size())
                #print(start_strip.size())
                
                padded_x = torch.cat((start_strip, padded_x, end_strip), dim=d)
                
                #print(padded_x.size())
        
        return padded_x
            

In [118]:
circ_pad_layer = CircularPadding(padding=[2,3])

In [119]:
a = np.arange(0,20)
a = a.reshape((1,5,4))
torch_a = torch.tensor(a)
print(torch_a)

tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11],
         [12, 13, 14, 15],
         [16, 17, 18, 19]]])


In [120]:
torch_a_pad = circ_pad_layer(torch_a)
print(torch_a_pad)

tensor([[[13, 14, 15, 12, 13, 14, 15, 12, 13, 14],
         [17, 18, 19, 16, 17, 18, 19, 16, 17, 18],
         [ 1,  2,  3,  0,  1,  2,  3,  0,  1,  2],
         [ 5,  6,  7,  4,  5,  6,  7,  4,  5,  6],
         [ 9, 10, 11,  8,  9, 10, 11,  8,  9, 10],
         [13, 14, 15, 12, 13, 14, 15, 12, 13, 14],
         [17, 18, 19, 16, 17, 18, 19, 16, 17, 18],
         [ 1,  2,  3,  0,  1,  2,  3,  0,  1,  2],
         [ 5,  6,  7,  4,  5,  6,  7,  4,  5,  6]]])


In [121]:
a = np.arange(0,20)
a = a.reshape((5,4))
torch_a = torch.tensor(a)
print(torch_a)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11],
        [12, 13, 14, 15],
        [16, 17, 18, 19]])


In [122]:
layer_pars = {"in_channels": 5,
              "out_channels": 10,
              "kernel_size": [2,2,2],
              "stride": [1,1,1],
              "padding": [0,0,0],
              "dilation": [1,1,1],
              "bias": True}
conv3d_layer = nn.Conv3d(**layer_pars)
#test_input = torch.ones(5,3,3,3)
test_input = torch.ones(1,5,3,3,3)
test_output = conv3d_layer(test_input)
print(test_input.size())
print(test_output.size())

torch.Size([1, 5, 3, 3, 3])
torch.Size([1, 10, 2, 2, 2])


In [135]:
"""4D Convolution"""

class MyConv4d(nn.Module):
    
    def __init__(self, in_channels=1, out_channels=1,
                 kernel_size=[1,1,1,1], stride=[1,1,1,1], padding=[0,0,0,0], padding_mode="zeros",
                 dilation=[1,1,1,1], groups=1,
                 bias=True,
                 kernel_initializer=None,
                 bias_initializer=None):
    
        #torch.nn.init.xavier_normal_
        super(MyConv4d, self).__init__()
        
        #save all arguments
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.padding_mode = padding_mode
        
        self.dilation = dilation
        self.groups = groups
        self.bias = bias
        
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        #(min_kernel_l, min_dims) = torch.min(self.kernel_size, 0, keepdim=True, out=None)
        
        #idea: separate a 4d convolution into a stack of 3d convolutions
        #intially the 4d kernel is stack split is done in the first dimension
        """CHANGE TO SHORTEST DIMENSION"""
        self.kernel_3d_num = self.kernel_size[0]
        self.kernel_3d_size = self.kernel_size[1:]
        
        conv_3d_layer_pars = {}
        conv_3d_layer_pars["in_channels"] = self.in_channels
        conv_3d_layer_pars["out_channels"] = self.out_channels
        
        conv_3d_layer_pars["kernel_size"] = self.kernel_3d_size
        conv_3d_layer_pars["stride"] = self.stride[1:]
        conv_3d_layer_pars["padding"] = self.padding[1:]
        conv_3d_layer_pars["padding_mode"] = self.padding_mode
        
        #conv3d_layer_pars["dilation"] = self.dilation[1:]
        #conv3d_layer_pars["groups"] = self.groups
        conv_3d_layer_pars["bias"] = self.bias
        
        #initialize conv_3d stack
        self.conv_3d_layers = nn.ModuleList()
        for kernel_3d_i in range(self.kernel_3d_num):
            #conv_3d_layer = nn.Conv3d(**conv3d_layer_pars)
            self.conv_3d_layers.append(nn.Conv3d(**conv_3d_layer_pars))
            
            if self.kernel_initializer != None:
                self.kernel_initializer(self.conv_3d_layers[-1].weight)
            if self.bias_initializer != None:
                self.bias_initializer(self.conv_3d_layers[-1].bias)
                
            

    def forward(self,x):
        
        x_size = np.array(x.size())
        print(f"input_size: {x_size}")
        
        #calculate the size of the output tensor
        #output_size = np.zeros(len(x_size), dtype=int)
        output_size = x_size.copy()
        """BATCH AXIS?"""
        #output_size[0] = x_size[0]
        #output_size[0] = self.out_channels
        output_size[1] = self.out_channels
        #new_layer_size_l = int(np.floor( (prev_layer_l + 2*padding_l - dilation_l*(kernel_l - 1) - 1)/(stride_l) + 1.))
        """GENERALIZE TO ARBITRARY DIMENSION"""
        for d in range(len(self.kernel_size)):
            #print(output_size[d+1])
            #print(x_size[d+1])
            #print(self.padding[d])
            #print(self.dilation[d])
            #print(self.kernel_size[d])
            #print(self.stride)
            output_size[d+2] = (x_size[d+2] + 2*self.padding[d] - self.dilation[d]*(self.kernel_size[d] - 1) - 1)/(self.stride[d]) + 1
        
        print(f"output_size: {output_size}")
        
        
        #keep track of the input row number in 4th dimension
        """CHANGE TO ARBITRARY DIMENSION"""
        #row_4d_num = x_size[1]
        row_4d_num = x_size[2]
        print(f"row_4d_num: {row_4d_num}")
        
        
        #keep track of the output row number in 4th dimension
        #output_row_4d_num = output_size[1]
        output_row_4d_num = output_size[2]
        print(f"output_row_4d_num: {output_row_4d_num}")
        
        
        #initialize output tensor
        output = torch.zeros(tuple(output_size))
        
        #iterate through every 3d slice of the 4d kernel
        for kernel_3d_i in range(self.kernel_3d_num):
            #print(f"kernel_3d_i: {kernel_3d_i}")
            
            """CAN BE REDUCED?"""
            #iterate through all rows in the fourth dimension
            #for row_4d_i in range(row_4d_num-self.kernel_3d_num):
            for row_4d_i in range(kernel_3d_i, row_4d_num - self.kernel_3d_num + kernel_3d_i):
            #for row_4d_i in range(row_4d_num):
                #print(f"row_4d_i: {row_4d_i}")
                
                print(f"kernel_3d_i: {kernel_3d_i}, row_4d_i: {row_4d_i}")
            
                """CHECK FORMULA"""
                output_row = row_4d_i - kernel_3d_i
                print(f"output_row: {output_row}")
                
                #check whether kernel slice is outside of the input tensor
                """IF CHECK NECESSARY?"""

                
                
                """GENERALIZE TO ARBITRARY DIMENSION"""
                #x_3d_slice = x[:,row_4d_i]
                x_3d_slice = x[:,:,row_4d_i]
                #print(f"x_3d_slice shape: {x_3d_slice.size()}")
                
                #compute the partial output row with the kernel slice
                #first two dimensions of x are batch size and channels
                """VIEW NECESSARY?"""
                partial_output_row = self.conv_3d_layers[kernel_3d_i](x_3d_slice)
                #frame_conv3d = self.conv3d_layers[i](input[:, :, j, :].view(b, c_i, d_i, h_i, w_i))
                
                """BATCH AND CHANNEL AXES?"""
                #output[:,output_row] += partial_output_row
                output[:,:,output_row] += partial_output_row
        
        
        return output
        
        #return output_size
        

In [136]:
layer_pars = {"in_channels": 5,
              "out_channels": 10,
              "kernel_size": [2,2,2,2],
              "stride": [1,1,1,1],
              "padding": [0,0,0,0],
              "dilation": [1,1,1,1],
              "bias": True}
conv4d_layer = MyConv4d(**layer_pars)


In [137]:
test_input = torch.ones(1,5,4,5,6,7)
test_output = conv4d_layer(test_input)
test_output.size()

input_size: [1 5 4 5 6 7]
output_size: [ 1 10  3  4  5  6]
row_4d_num: 4
output_row_4d_num: 3
kernel_3d_i: 0, row_4d_i: 0
output_row: 0
kernel_3d_i: 0, row_4d_i: 1
output_row: 1
kernel_3d_i: 1, row_4d_i: 1
output_row: 0
kernel_3d_i: 1, row_4d_i: 2
output_row: 1


torch.Size([1, 10, 3, 4, 5, 6])

In [138]:
"""4D Convolution v2"""
"""No padding support due to convolution algorithm"""
"""
Currently works only with [batch_index, channel_index, t, x, y, z] format
the four last index blocks are convolved
the 3d slices are cut with respect to the t-axis
"""

class MyConv4dv2(nn.Module):
    
    def __init__(self, in_channels=1, out_channels=1,
                 kernel_size=[1,1,1,1], stride=[1,1,1,1],
                 dilation=[1,1,1,1], groups=1, padding=[0,0,0,0], padding_mode="zeros",
                 bias=True,
                 kernel_initializer=None,
                 bias_initializer=None):
    
        #torch.nn.init.xavier_normal_
        super(MyConv4dv2, self).__init__()
        
        #save all arguments
        self.in_channels = in_channels
        self.out_channels = out_channels
        
        self.kernel_size = kernel_size
        self.stride = stride
        self.padding = padding
        self.padding_mode = padding_mode
        
        self.dilation = dilation
        self.groups = groups
        self.bias = bias
        
        self.kernel_initializer = kernel_initializer
        self.bias_initializer = bias_initializer
        
        #(min_kernel_l, min_dims) = torch.min(self.kernel_size, 0, keepdim=True, out=None)
        
        #idea: separate a 4d convolution into a stack of 3d convolutions
        #intially the 4d kernel is stack split is done in the first dimension
        """CHANGE TO SHORTEST DIMENSION"""
        self.kernel_3d_num = self.kernel_size[0]
        self.kernel_3d_size = self.kernel_size[1:]
        
        self.padding_4d = self.padding[0]
        
        conv_3d_layer_pars = {}
        conv_3d_layer_pars["in_channels"] = self.in_channels
        conv_3d_layer_pars["out_channels"] = self.out_channels
        
        conv_3d_layer_pars["kernel_size"] = self.kernel_3d_size
        """NOT YET IMPLEMENTED"""
        conv_3d_layer_pars["stride"] = self.stride[1:]
        conv_3d_layer_pars["padding"] = self.padding[1:]
        #conv_3d_layer_pars["padding_mode"] = self.padding_mode
        
        """NOT YET IMPLEMENTED"""
        #conv_3d_layer_pars["dilation"] = self.dilation[1:]
        #conv_3d_layer_pars["groups"] = self.groups
        conv_3d_layer_pars["bias"] = self.bias
        
        #initialize conv_3d stack
        self.conv_3d_layers = nn.ModuleList()
        for kernel_3d_i in range(self.kernel_3d_num):
            #conv_3d_layer = nn.Conv3d(**conv3d_layer_pars)
            self.conv_3d_layers.append(nn.Conv3d(**conv_3d_layer_pars))
            
            if self.kernel_initializer != None:
                self.kernel_initializer(self.conv_3d_layers[-1].weight)
            if self.bias_initializer != None:
                self.bias_initializer(self.conv_3d_layers[-1].bias)
                
            

    def forward(self,x):
        
        x_size = np.array(x.size())
        print(f"input_size: {x_size}")
        
        #calculate the size of the output tensor
        #output_size = np.zeros(len(x_size), dtype=int)
        output_size = x_size.copy()
        """BATCH AXIS?"""
        #output_size[0] = x_size[0]
        #output_size[0] = self.out_channels
        output_size[1] = self.out_channels
        #new_layer_size_l = int(np.floor( (prev_layer_l + 2*padding_l - dilation_l*(kernel_l - 1) - 1)/(stride_l) + 1.))
        """GENERALIZE TO ARBITRARY DIMENSION"""
        for d in range(len(self.kernel_size)):
            #print(output_size[d+1])
            #print(x_size[d+1])
            #print(self.padding[d])
            #print(self.dilation[d])
            #print(self.kernel_size[d])
            #print(self.stride)
            output_size[d+2] = (x_size[d+2] + 2*self.padding[d] - self.dilation[d]*(self.kernel_size[d] - 1) - 1)/(self.stride[d]) + 1
        
        #print(f"output_size: {output_size}")
        
        
        #keep track of the input row number in 4th dimension
        #disregarding padding!
        """CHANGE TO ARBITRARY DIMENSION"""
        #row_4d_num = x_size[1]
        row_4d_num = x_size[2]
        #print(f"row_4d_num: {row_4d_num}")
        
        
        #keep track of the output row number in 4th dimension
        #output_row_4d_num = output_size[1]
        output_row_4d_num = output_size[2]
        #print(f"output_row_4d_num: {output_row_4d_num}")
        
        
        #initialize output tensor
        output = torch.zeros(tuple(output_size))
        
        #iterate through every 3d slice of the 4d kernel
        for kernel_3d_i in range(self.kernel_3d_num):
            #print(f"kernel_3d_i: {kernel_3d_i}")
            
            """CHECK FORMULA"""
            #perform a convolution with current 3d kernel slice on ALL of these rows
            rows = [kernel_3d_i, row_4d_num - self.kernel_3d_num + kernel_3d_i + 1]
            
            #print(f"convolving rows: [{rows[0]},{rows[1]})")
            
            #"""CHECK FORMULA"""
            #output_row = row_4d_i - kernel_3d_i
            #print(f"output_row: {output_row}")
            
            #the block with which the 3d conv slice should be convolved
            """GENERALIZE TO ARBITRARY DIMENSION"""
            x_4d_block = x.narrow(2, int(rows[0]), int(rows[1] - rows[0]))
            
            #print(f"slicing 4d block of size: {x_4d_block.size()}")
            
            #swap row axis and channel axis
            #so that convolution can sum up channels
            #but leave rows in 4d untouched
            #x_4d_block.transpose(2,1)
            x_4d_block_t = torch.transpose(x_4d_block, 1, 2)
            #print(f"transposing 4d block to size: {x_4d_block_t.size()}")
            
            #perform the actual operation
            #the input tensor has to have 5 dimensions in order for conv3d to work
            #squeeze the first two dimensions into the batch dimension
            #unsqueezing has to be done later
            #block_output = self.conv3d_layers[kernel_3d_i](x_4d_block_t.view(-1,*x_4d_block_t.size()[2:]))
            #print(f"convolving 4d block of size: {x_4d_block_t.view(-1,*x_4d_block_t.size()[2:]).size()}")
            
            block_output = self.conv_3d_layers[kernel_3d_i](x_4d_block_t.reshape(-1,*x_4d_block_t.size()[2:]))
            #print(f"convolving 4d block of size: {x_4d_block_t.reshape(-1,*x_4d_block_t.size()[2:]).size()}")

            #aggregate the block_output
            """UNSQUEEZING BATCH AND ROW DIMENSION CORRECT?"""
            #output[:,:] += block_output
            output += block_output.view(*output_size)

        
        
        return output

In [139]:
layer_pars = {"in_channels": 5,
              "out_channels": 10,
              "kernel_size": [3,3,3,3],
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True}
conv4dv2_layer = MyConv4dv2(**layer_pars)

In [145]:
test_input = torch.ones(2,5,6,8,8,8)
#test_input.transpose()
#test_input.reshape()
test_output = conv4dv2_layer(test_input)
test_output.size()

input_size: [2 5 6 8 8 8]
output_size: [ 2 10  4  6  6  6]
row_4d_num: 6
output_row_4d_num: 4
kernel_3d_i: 0
convolving rows: [0,4)
slicing 4d block of size: torch.Size([2, 5, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([2, 4, 5, 8, 8, 8])
convolving 4d block of size: torch.Size([8, 5, 8, 8, 8])
kernel_3d_i: 1
convolving rows: [1,5)
slicing 4d block of size: torch.Size([2, 5, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([2, 4, 5, 8, 8, 8])
convolving 4d block of size: torch.Size([8, 5, 8, 8, 8])
kernel_3d_i: 2
convolving rows: [2,6)
slicing 4d block of size: torch.Size([2, 5, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([2, 4, 5, 8, 8, 8])
convolving 4d block of size: torch.Size([8, 5, 8, 8, 8])


torch.Size([2, 10, 4, 6, 6, 6])

In [165]:
kernel_size = [3,4,5,6]
layer_pars = {"in_channels": 1,
              "out_channels": 1,
              "kernel_size": kernel_size,
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True,
              "kernel_initializer" : lambda x: torch.nn.init.constant_(x, 1),
              "bias_initializer" : lambda x: torch.nn.init.constant_(x, 0)
             }
#kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
#bias_initializer=lambda x: torch.nn.init.constant_(x, 0))

conv4dv2_layer = MyConv4dv2(**layer_pars)

test_input_shape = [1,1,6,8,8,8]

test_input = torch.ones(*test_input_shape)

test_output = conv4dv2_layer(test_input)
print(test_output.size())
print(test_output)

input_size: [1 1 6 8 8 8]
output_size: [1 1 4 5 4 3]
row_4d_num: 6
output_row_4d_num: 4
kernel_3d_i: 0
convolving rows: [0,4)
slicing 4d block of size: torch.Size([1, 1, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 1, 8, 8, 8])
kernel_3d_i: 1
convolving rows: [1,5)
slicing 4d block of size: torch.Size([1, 1, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 1, 8, 8, 8])
kernel_3d_i: 2
convolving rows: [2,6)
slicing 4d block of size: torch.Size([1, 1, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 1, 8, 8, 8])
torch.Size([1, 1, 4, 5, 4, 3])
tensor([[[[[[360., 360., 360.],
            [360., 360., 360.],
            [360., 360., 360.],
            [360., 360., 360.]],

           [[360., 360., 360.],
            [360., 360., 360.],
            [360., 360., 360.],
            [360

In [197]:
kernel_size = [3,4,5,6]
layer_pars = {"in_channels": 3,
              "out_channels": 1,
              "kernel_size": kernel_size,
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True,
              "kernel_initializer" : lambda x: torch.nn.init.constant_(x, 1),
              "bias_initializer" : lambda x: torch.nn.init.constant_(x, 0)
             }
#kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
#bias_initializer=lambda x: torch.nn.init.constant_(x, 0))

conv4dv2_layer = MyConv4dv2(**layer_pars)

test_input_shape = [1,3,6,8,8,8]

test_input = torch.ones(*test_input_shape)


test_output = conv4dv2_layer(test_input)
print(test_output.size())
print(test_output)

input_size: [1 3 6 8 8 8]
output_size: [1 1 4 5 4 3]
row_4d_num: 6
output_row_4d_num: 4
kernel_3d_i: 0
convolving rows: [0,4)
slicing 4d block of size: torch.Size([1, 3, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 3, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 3, 8, 8, 8])
kernel_3d_i: 1
convolving rows: [1,5)
slicing 4d block of size: torch.Size([1, 3, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 3, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 3, 8, 8, 8])
kernel_3d_i: 2
convolving rows: [2,6)
slicing 4d block of size: torch.Size([1, 3, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 3, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 3, 8, 8, 8])
torch.Size([1, 1, 4, 5, 4, 3])
tensor([[[[[[1080., 1080., 1080.],
            [1080., 1080., 1080.],
            [1080., 1080., 1080.],
            [1080., 1080., 1080.]],

           [[1080., 1080., 1080.],
            [1080., 1080., 1080.],
            [1080., 1080., 108

In [167]:
kernel_size = [3,4,5,6]
layer_pars = {"in_channels": 1,
              "out_channels": 1,
              "kernel_size": kernel_size,
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True,
              "kernel_initializer" : lambda x: torch.nn.init.constant_(x, 2),
              "bias_initializer" : lambda x: torch.nn.init.constant_(x, 0)
             }
#kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
#bias_initializer=lambda x: torch.nn.init.constant_(x, 0))

conv4dv2_layer = MyConv4dv2(**layer_pars)

test_input_shape = [1,1,6,8,8,8]

test_input = torch.ones(*test_input_shape)

test_output = conv4dv2_layer(test_input)
print(test_output.size())
print(test_output)

input_size: [1 1 6 8 8 8]
output_size: [1 1 4 5 4 3]
row_4d_num: 6
output_row_4d_num: 4
kernel_3d_i: 0
convolving rows: [0,4)
slicing 4d block of size: torch.Size([1, 1, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 1, 8, 8, 8])
kernel_3d_i: 1
convolving rows: [1,5)
slicing 4d block of size: torch.Size([1, 1, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 1, 8, 8, 8])
kernel_3d_i: 2
convolving rows: [2,6)
slicing 4d block of size: torch.Size([1, 1, 4, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 4, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([4, 1, 8, 8, 8])
torch.Size([1, 1, 4, 5, 4, 3])
tensor([[[[[[720., 720., 720.],
            [720., 720., 720.],
            [720., 720., 720.],
            [720., 720., 720.]],

           [[720., 720., 720.],
            [720., 720., 720.],
            [720., 720., 720.],
            [720

In [251]:
kernel_size = [3,4,5,6]
layer_pars = {"in_channels": 1,
              "out_channels": 1,
              "kernel_size": kernel_size,
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True,
              "kernel_initializer" : lambda x: torch.nn.init.constant_(x, 1),
              "bias_initializer" : lambda x: torch.nn.init.constant_(x, 1)
             }
#kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
#bias_initializer=lambda x: torch.nn.init.constant_(x, 0))

conv4dv2_layer = MyConv4dv2(**layer_pars)

test_input_shape = [1,1,6,8,8,8]

test_input = torch.ones(*test_input_shape)

test_output = conv4dv2_layer(test_input)
print(test_output.size())
print(test_output)

input_size: [1 1 6 8 8 8]
output_size: [1 1 3 5 4 3]
row_4d_num: 6
output_row_4d_num: 3
kernel_3d_i: 0
convolving rows: [0,3)
slicing 4d block of size: torch.Size([1, 1, 3, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 3, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([3, 1, 8, 8, 8])
kernel_3d_i: 1
convolving rows: [1,4)
slicing 4d block of size: torch.Size([1, 1, 3, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 3, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([3, 1, 8, 8, 8])
kernel_3d_i: 2
convolving rows: [2,5)
slicing 4d block of size: torch.Size([1, 1, 3, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 3, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([3, 1, 8, 8, 8])
kernel_3d_i: 3
convolving rows: [3,6)
slicing 4d block of size: torch.Size([1, 1, 3, 8, 8, 8])
transposing 4d block to size: torch.Size([1, 3, 1, 8, 8, 8])
convolving 4d block of size: torch.Size([3, 1, 8, 8, 8])
torch.Size([1, 1, 3, 5, 4, 3])
tensor([[[[[[484., 484., 484.

In [250]:
kernel_size = [3,4,5,6]
conv4d_layer = \
        Conv4d(in_channels=1,
               out_channels=1,
               kernel_size=kernel_size,
               padding=0,
               kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
               bias_initializer=lambda x: torch.nn.init.constant_(x, 1))
    
test_input_shape = [1,1,6,8,8,8]

test_input = torch.ones(*test_input_shape)

test_output = conv4d_layer(test_input)
print(test_output.size())
print(test_output)

torch.Size([1, 1, 3, 5, 4, 3])
tensor([[[[[[363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.]],

           [[363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.]],

           [[363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.]],

           [[363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.]],

           [[363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.],
            [363., 363., 363.]]],


          [[[484., 484., 484.],
            [484., 484., 484.],
            [484., 484., 484.],
            [484., 484., 484.]],

           [[484., 484., 484.],
            [484., 484., 484.],
            [484., 484., 484.],
            [484., 484., 484.]],

           [[484., 484., 484.],
            [484., 484., 

In [252]:
kernel_size = [3,3,3,3]
layer_pars = {"in_channels": 1,
              "out_channels": 1,
              "kernel_size": kernel_size,
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True,
              "kernel_initializer" : lambda x: torch.nn.init.constant_(x, 1),
              "bias_initializer" : lambda x: torch.nn.init.constant_(x, 0)
             }
#kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
#bias_initializer=lambda x: torch.nn.init.constant_(x, 0))

conv4dv2_layer = MyConv4dv2(**layer_pars)

test_input_shape = [1,1,5,5,5,5]

test_input = torch.zeros(*test_input_shape)
#test_input[0,0,1,:,:,:] = 1
#test_input[0,0,0,:,:,:] = 1

for i in range(0,test_input_shape[2]):
    test_input[0,0,i,:,:,:] = i
print(test_input)
test_output = conv4dv2_layer(test_input)
print(test_output.size())
print(test_output)

tensor([[[[[[0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.]],

           [[0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.]],

           [[0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.]],

           [[0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.]],

           [[0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0.]]],


          [[[1., 1., 1., 1., 1.],
            [1., 1., 1., 1., 1.],
            [1., 1., 1., 1., 1.],
            [1., 1., 1., 1., 1.],
  

In [235]:
kernel_size = [2,2,2,2]
layer_pars = {"in_channels": 1,
              "out_channels": 1,
              "kernel_size": kernel_size,
              "stride": [1,1,1,1],
              "dilation": [1,1,1,1],
              "bias": True,
              "kernel_initializer" : lambda x: torch.nn.init.constant_(x, 1),
              "bias_initializer" : lambda x: torch.nn.init.constant_(x, 0)
             }
#kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
#bias_initializer=lambda x: torch.nn.init.constant_(x, 0))

conv4dv2_layer = MyConv4dv2(**layer_pars)

test_input_shape = [1,1,3,3,3,3]

test_input = np.arange(float(np.prod(test_input_shape)))
test_input = test_input.reshape(test_input_shape)
test_input = torch.tensor(test_input)
print(test_input)
test_output = conv4dv2_layer(test_input.float())
print(test_output.size())
print(test_output)

tensor([[[[[[ 0.,  1.,  2.],
            [ 3.,  4.,  5.],
            [ 6.,  7.,  8.]],

           [[ 9., 10., 11.],
            [12., 13., 14.],
            [15., 16., 17.]],

           [[18., 19., 20.],
            [21., 22., 23.],
            [24., 25., 26.]]],


          [[[27., 28., 29.],
            [30., 31., 32.],
            [33., 34., 35.]],

           [[36., 37., 38.],
            [39., 40., 41.],
            [42., 43., 44.]],

           [[45., 46., 47.],
            [48., 49., 50.],
            [51., 52., 53.]]],


          [[[54., 55., 56.],
            [57., 58., 59.],
            [60., 61., 62.]],

           [[63., 64., 65.],
            [66., 67., 68.],
            [69., 70., 71.]],

           [[72., 73., 74.],
            [75., 76., 77.],
            [78., 79., 80.]]]]]], dtype=torch.float64)
input_size: [1 1 3 3 3 3]
output_size: [1 1 2 2 2 2]
row_4d_num: 3
output_row_4d_num: 2
kernel_3d_i: 0
convolving rows: [0,2)
slicing 4d block of size: torch.Size([1, 1, 2

In [245]:
test_input[0,0,0:2,0:2,0:2,0:2].sum()

tensor(320., dtype=torch.float64)

In [ ]:
test_input[0,0,0:2,1:3,1:3,1:3].sum()

In [179]:
def conv4d_check():

    print()
    print('TEST PYTORCH CONV4D LAYER IMPLEMENTATION')
    print('\n' + 80 * '-' + '\n')

    # -------------------------------------------------------------------------
    # Generate random input 4D tensor (+ batch dimension, + channel dimension)
    # -------------------------------------------------------------------------

    np.random.seed(42)

    input_numpy = np.round(np.random.random((1, 1, 10, 11, 12, 13)) * 100)
    input_torch = torch.from_numpy(input_numpy).float()

    # -------------------------------------------------------------------------
    # Convolve with a randomly initialized kernel
    # -------------------------------------------------------------------------

    print('Randomly Initialized Kernels:\n')

    # Initialize the 4D convolutional layer with random kernels
    conv4d_layer = \
        Conv4d(in_channels=1,
               out_channels=1,
               kernel_size=(3, 3, 3, 3)
               )
    
    myconv4d_layer = \
        MyConv4d(in_channels=1,
               out_channels=1,
               kernel_size=(3, 3, 3, 3)
               )
    
    myconv4dv2_layer = \
        MyConv4dv2(in_channels=1,
               out_channels=1,
               kernel_size=(3, 3, 3, 3)
               )

    # Pass the input tensor through that layer
    output = conv4d_layer.forward(input_torch).data.numpy()
    myoutput = myconv4d_layer.forward(input_torch).data.numpy()
    myoutputv2 = myconv4dv2_layer.forward(input_torch).data.numpy()

    # Select the 3D kernels for the manual computation and comparison
    kernels = [conv4d_layer.conv3d_layers[i].weight.data.numpy().flatten()
               for i in range(3)]

    # Compare the conv4d_layer result and the manual convolution computation
    # at 3 randomly chosen locations
    for i in range(3):

        # Randomly choose a location and select the conv4d_layer output
        loc = [np.random.randint(0, output.shape[2] - 2),
               np.random.randint(0, output.shape[3] - 2),
               np.random.randint(0, output.shape[4] - 2),
               np.random.randint(0, output.shape[5] - 2)]
        conv4d = output[0, 0, loc[0], loc[1], loc[2], loc[3]]
        myconv4d = myoutput[0, 0, loc[0], loc[1], loc[2], loc[3]]
        myconv4dv2 = myoutputv2[0, 0, loc[0], loc[1], loc[2], loc[3]]

        # Select slices from the input tensor and compute manual convolution
        slices = [input_numpy[0, 0, loc[0] + j, loc[1]:loc[1] + 3,
                              loc[2]:loc[2] + 3, loc[3]:loc[3] + 3].flatten()
                  for j in range(3)]
        manual = np.sum([slices[j] * kernels[j] for j in range(3)])

        # Print comparison
        print(f'At {tuple(loc)}:')
        print(f'\tconv4d:\t{conv4d}')
        print(f'\tmyconv4d:\t{myconv4d}')
        print(f'\tmyconv4dv2:\t{myconv4dv2}')
        print(f'\tmanual:\t{manual}')

    print('\n' + 80 * '-' + '\n')

    # -------------------------------------------------------------------------
    # Convolve with a kernel initialized to be all ones
    # -------------------------------------------------------------------------

    print('Constant Kernels (all 1):\n')

    conv4d_layer = \
        Conv4d(in_channels=1,
               out_channels=1,
               kernel_size=(3, 3, 3, 3),
               padding=1,
               kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
               bias_initializer=lambda x: torch.nn.init.constant_(x, 0))
    output = conv4d_layer.forward(input_torch)
    
    myconv4d_layer = \
        MyConv4d(in_channels=1,
               out_channels=1,
               kernel_size=(3, 3, 3, 3),
               #padding=1,
               kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
               bias_initializer=lambda x: torch.nn.init.constant_(x, 0))
    output = conv4d_layer.forward(input_torch)
    
    myconv4dv2_layer = \
        MyConv4dv2(in_channels=1,
               out_channels=1,
               kernel_size=(3, 3, 3, 3),
               #padding=1,
               kernel_initializer=lambda x: torch.nn.init.constant_(x, 1),
               bias_initializer=lambda x: torch.nn.init.constant_(x, 0))
    
    output = conv4d_layer.forward(input_torch)
    myoutput = myconv4d_layer.forward(input_torch)
    myoutputv2 = myconv4dv2_layer.forward(input_torch)
    
    # Define relu(x) = max(x, 0) for simplified indexing below
    def relu(x: float) -> float:
        return x * (x > 0)

    # Compare the conv4d_layer result and the manual convolution computation
    # at 3 randomly chosen locations
    for i in range(3):

        # Randomly choose a location and select the conv4d_layer output
        loc = [np.random.randint(0, output.shape[2] - 2),
               np.random.randint(0, output.shape[3] - 2),
               np.random.randint(0, output.shape[4] - 2),
               np.random.randint(0, output.shape[5] - 2)]
        conv4d = output[0, 0, loc[0], loc[1], loc[2], loc[3]]
        myconv4d = myoutput[0, 0, loc[0], loc[1], loc[2], loc[3]]
        myconv4dv2 = myoutputv2[0, 0, loc[0], loc[1], loc[2], loc[3]]

        # For a kernel that is all 1s, we only need to sum up the elements of
        # the input (the ReLU takes care of the padding!)
        manual = input_numpy[0, 0,
                             relu(loc[0] - 1):loc[0] + 2,
                             relu(loc[1] - 1):loc[1] + 2,
                             relu(loc[2] - 1):loc[2] + 2,
                             relu(loc[3] - 1):loc[3] + 2].sum()

        # Print comparison
        print(f'At {tuple(loc)}:')
        print(f'\tconv4d:\t{conv4d}')
        print(f'\tmyconv4d:\t{myconv4d}')
        print(f'\tmyconv4dv2:\t{myconv4dv2}')
        print(f'\tmanual:\t{manual}')

    print()

In [144]:
conv4d_check()


TEST PYTORCH CONV4D LAYER IMPLEMENTATION

--------------------------------------------------------------------------------

Randomly Initialized Kernels:

input_size: [ 1  1 10 11 12 13]
output_size: [ 1  1  8  9 10 11]
row_4d_num: 10
output_row_4d_num: 8
kernel_3d_i: 0, row_4d_i: 0
output_row: 0
kernel_3d_i: 0, row_4d_i: 1
output_row: 1
kernel_3d_i: 0, row_4d_i: 2
output_row: 2
kernel_3d_i: 0, row_4d_i: 3
output_row: 3
kernel_3d_i: 0, row_4d_i: 4
output_row: 4
kernel_3d_i: 0, row_4d_i: 5
output_row: 5
kernel_3d_i: 0, row_4d_i: 6
output_row: 6
kernel_3d_i: 1, row_4d_i: 1
output_row: 0
kernel_3d_i: 1, row_4d_i: 2
output_row: 1
kernel_3d_i: 1, row_4d_i: 3
output_row: 2
kernel_3d_i: 1, row_4d_i: 4
output_row: 3
kernel_3d_i: 1, row_4d_i: 5
output_row: 4
kernel_3d_i: 1, row_4d_i: 6
output_row: 5
kernel_3d_i: 1, row_4d_i: 7
output_row: 6
kernel_3d_i: 2, row_4d_i: 2
output_row: 0
kernel_3d_i: 2, row_4d_i: 3
output_row: 1
kernel_3d_i: 2, row_4d_i: 4
output_row: 2
kernel_3d_i: 2, row_4d_i: 5
o